In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx

from shapely import (
    Point,
    LineString,
    union_all,
    ops,
)
from pyproj import (
    Transformer,
)

import os
import warnings

from typing import (
    List,
    Tuple,
    Dict,
    Any,
    Optional,
)

from collections.abc import (
    Collection,
    Callable,
)

In [2]:
# MERIT-Basins path
mb_path = '/project/rrg-mclark/data/geospatial-data/MERIT-Basins/MERIT_Hydro_v07_Basins_v01_bugfix1/'

# cat layer
cat = gpd.read_file(os.path.join(mb_path, 'pfaf_level_02', 'cat_pfaf_72_MERIT_Hydro_v07_Basins_v01_bugfix1.shp'))

# riv layer
riv = gpd.read_file(os.path.join(mb_path, 'pfaf_level_02', 'riv_pfaf_72_MERIT_Hydro_v07_Basins_v01_bugfix1.shp'))

# cst layer
cst = gpd.read_file(os.path.join(mb_path, 'coastal_hillslopes', 'hillslope_72_clean.shp'))

In [3]:
successor = 72051873
predecessor = 72053625

# user inputs
attr = 'lengthkm'
main_id = 'COMID'
ds_main_id = 'NextDownID'

# build a networkx DiGraph out of GeoPandas GeoDataFrame of a river network
rg = nx.from_pandas_edgelist(df=riv,
                             source=main_id,
                             target=ds_main_id,
                             edge_attr=True,
                             create_using=nx.DiGraph)

# each river confluence (networkx node) corresponds to its outgoing edge
# (river segment), and all attributes are similar for both objects
nx.set_node_attributes(
    G=rg,
    values=riv.set_index(main_id, drop=False).T.to_dict()
)

In [4]:
s = rg.nodes[successor]
p = rg.nodes[predecessor]

____

Spatial distance method:

In [5]:
def _aggregate_linestring_geoms(
    successor: LineString,
    predecessor: LineString,
) -> LineString:
    """Aggregated (Multi-)LineString geometries for `successor` and
    `predecessor` objects
    
    Parameters
    ----------
    successor: shapely.geometry.LineString
        sucessor LineString to be aggregated
    predecessor: shapely.geometry.LineString
        predecessor LineString to be aggregated with `successor`
    
    Returns
    -------
    shapely.LineString:
        aggregated LineString geometry
    """
    # check if successor and predecessor are shapely.geometry.LineString types
    if not isinstance(successor, LineString):
        raise TypeError("`successor` must be of type shapely.geometry.LineString")
    if not isinstance(predecessor, LineString):
        raise TypeError("`predecessor` must be of type shapely.geometry.LineString")
        
    return ops.linemerge(union_all([successor, predecessor]))

def linestrings_endpoint_distance(
    successor: LineString,
    predecessor: LineString,
    linestring_epsg: Optional[int] = 4326,
    distance_epsg: Optional[int] = 6933,
    *args,
    **kwargs,
) -> float:
    """Calculate the horizontal direct distance between the end points of
    two connected shapely.LineString objects. 
    
    Parameters
    ----------
    successor : shapely.LineString
        The successer river segment that shares a point(s) with the
        `predecessor` object.
    predecessor : shapely.LineString
        The predecessor to `successor` object shareing a common point(s).
    linestring_epsg : int, optional [defaults to `4326`]
        the source EPSG code describing the `crs` of both the `successor` and
        `predecessor` objects. Either `epsg` or `crs` options could be used
        but not both.
    distance_epsg : int, optional [defailts to `6933`]
        the target EPSG code used to calculate the distance between the
        coordinates.
    
    Returns
    -------
    float
        The horizontal distance between the endpoints of two given
        shapely.LineString objects. The unit of the returned value is in
        `meters`, if the `distance_epsg` is set to its default value of
        `6933`.
    
    Raises
    ------
    ValueError
        If `successor` and `predecessor` do not share a common point(s), this
        exception will be raised.
    """
    # check the dtype of `successor` and `predecessor`
    for v in [successor, predecessor]:
        if not isinstance(v, LineString):
            raise ValueError("Two given LineStrings do not touch")

    # check if there is a common point (coordinate) shared between
    # `successor` and `predecessor`
    shared_point = set(successor.coords).intersection(set(predecessor.coords))
    if not shared_point:
        raise ValueError("A common point shared between LineString "
                         "objects is missing")

    # aggregate the geometries of LineString objects
    agg_line = _aggregate_linestring_geoms(successor, predecessor)

    # extract the end points
    end_points = (
        Point(agg_line.coords[0]), 
        Point(agg_line.coords[-1])
    )

    # transform the given `linestring_EPSG` to an equal area 
    # `distance_epsg`
    transformer = Transformer.from_crs(
        linestring_epsg,
        distance_epsg,
        always_xy=True,
    )
    
    # equal-area projection transformed end_points
    transformed_end_points = \
        [Point(transformer.transform(geom.x, geom.y))
             for geom in end_points]

    # calculate distance
    # return end_points
    return _coords_spatial_distance(
        transformed_end_points[0],
        transformed_end_points[1],
    )

def _coords_spatial_distance(
    p1: Point,
    p2: Point,
) -> float:
    """Return the distance between `p1` and `p2`
    
    Parameters
    ----------
    p1 : shapely.Point
        Coordinates of the first point
    p2 : shapely.Point
        Coordinates of the second point
        
    Returns
    -------
    float
        The distance (2D or 3D, depending on Point data) between `p1` and `p2`
    """

    return p1.distance(p2)

In [6]:
linestrings_endpoint_distance(
    successor=s['geometry'],
    predecessor=p['geometry'],
    linestring_epsg=4326,
    distance_epsg=6933
)

1221.3122269768437

____

upstream and downstream values

In [8]:
def upstream_attr(
    successor: Any,
    predecessor: Any,
    *args,
    **kwargs,
) -> Any:
    """Return the predecessor's attribute
    
    Parameters
    ----------
    successor : Any
        successor attribute value to be ignored
    predecessor : Any
        predecessor attribute value to be returned
    
    Returns
    -------
    predecessor : Any
        attribute value to be returned
    """
    successor = None
    
    if not predecessor:
        warnings.warn(f"`predecessor` is set to {predecessor}")
    
    return predecessor

In [9]:
def downstream_attr(
    successor: Any,
    predecessor: Any = None,
    *args,
    **kwargs,
) -> Any:
    """Return the predecessor's attribute
    
    Parameters
    ----------
    successor : Any
        successor attribute value to be returned
    predecessor : Any
        predecessor attribute value to be ignored
    
    Returns
    -------
    successor : Any
        attribute value to be returned
    """
    
    predecessor = None
    
    if successor is None:
        warnings.warn("`successor` is set to None")
    
    return successor

weighted averages

In [10]:
def weighted_mean(
    succesor: Any,
    predecessor: Any,
    successor_weight: float,
    predecessor_weight: float,
    *args,
    **kwargs,
) -> Any:
    """Weighted average of `successor` and `predecessor` values given the
    `successor_weight` and `predecessor_weight` weights
    
    Parameters
    ----------
    successor : Any
        successor segment attribute value
    predecessor : Any
        predecessor segment attribute value
    successor_weight : float
        weight for the `successor`
    predecessor_weight : float
        weight for the `predecessor`
    
    Returns
    -------
    float | int
        weighted mean of input arguments
    """
    numerator = (succesor * successor_weight) + (predecessor * predecessor_weight)
    denominator = (successor_weight + predecessor_weight)
    
    return numerator / denominator

____